In [1]:
import pandas as pd
import plotly.express as px


data = pd.read_csv("../input/fortune-global-2000-companies-till-2021/fortune_2000_in_2021.csv")
data.head()

## Get numeric values for Sales, Profit, Assets & Market Value

In [1]:
def convert_to_numeric(col: str, data: pd.DataFrame = data) -> pd.Series:
    """Get numeric values for the specified `col` i.e convert '$1M to 1000000.

    Parameters
    ----------
    col : str
        The name of a column in the dataset `data`.
    data : pandas.core.frame.DataFrame
        A dataset having columns with numeric values stored as strings.

    Returns
    -------
    pandas.core.series.Series
        Numeric values."""
    scale = data[col].str.contains("M").map({True: 1e6, False: 1e9})
    digits = (
        data[col]
        .str.replace(",", "")
        .str.extract("(\d+\.?\d*)", expand=False)
        .astype("float")
    )
    return digits * scale


numeric_columns = ["Assets", "Market Value", "Sales", "Profit"]

for col in numeric_columns:
    data[col] = convert_to_numeric(col)

data.head()

# Contribution of the Various Components

In [1]:
colors = ["skyblue", "navy", "steelblue", "#55ee55"]
n = 10
top_n = data.nsmallest(n, columns="Rank")


bar_chart = px.bar(
    top_n,
    x="Name",
    y=numeric_columns,
    barmode="stack",
    color_discrete_sequence=colors,
    height=550,
    width=800,
    title=f"Top {n} Companies",
    template="simple_white",
)
bar_chart.update_layout(
    font_family="serif", title_font_size=20, title_x=0.5, hovermode="x unified"
)
bar_chart.update_traces(hovertemplate="%{y}")
bar_chart.update_xaxes(fixedrange=True, tickangle=-45, title="Company")
bar_chart.update_yaxes(fixedrange=True, title="Value")
bar_chart.show()


pie_chart = px.pie(
    names=numeric_columns,
    values=data[numeric_columns].sum(),
    color_discrete_sequence=colors,
    hole=0.5,
    height=480,
    width=800,
    title="Overall (Sum)",
)
pie_chart.update_traces(hovertemplate="<i>%{label}</i>: <b>%{value:.3s}</b>")
pie_chart.update_layout(font_family="serif", title_font_size=20, title_x=0.5)
pie_chart.show()

# Performance of Top Ranking Companies

Making the most sales or profit doesn't necessarily guarantee the highest ranking.

In [1]:
fig = px.bar(
    top_n,
    x="Name",
    y=["Sales", "Profit"],
    barmode="stack",
    color_discrete_sequence=["steelblue", "#55ee55"],
    height=550,
    width=800,
    title=f"Performance of the Top {n} Companies",
    template="simple_white",
)
fig.update_layout(
    font_family="serif", title_font_size=20, title_x=0.5, hovermode="x unified"
)
fig.update_traces(hovertemplate="%{y}")
fig.update_xaxes(fixedrange=True, tickangle=-45, title="Company")
fig.update_yaxes(fixedrange=True, title="Value")

# Value of Top Ranking Companies

It is rather interesting that the highest ranked company has a lower market value than its peers. But it more than makes up for it by having the largest asset volume.

In [1]:
fig = px.bar(
    top_n,
    x="Name",
    y=["Market Value", "Assets"],
    barmode="stack",
    color_discrete_sequence=["blue", "skyblue"],
    height=550,
    width=800,
    title=f"Value of the Top {n} Companies",
    template="simple_white",
)
fig.update_layout(
    font_family="serif", title_font_size=20, title_x=0.5, hovermode="x unified"
)
fig.update_traces(hovertemplate="%{y}")
fig.update_xaxes(fixedrange=True, tickangle=-45, title="Company")
fig.update_yaxes(fixedrange=True, title="Value")

# Relationship between Rank and Performance, Value

In [1]:
correlation_with_rank = data.corrwith(data["Rank"])


for column, color in zip(numeric_columns, colors):
    fig = px.scatter(
        data.nsmallest(100, columns="Rank"),
        x="Rank",
        y=column,
        color="Rank",
        color_continuous_scale=[color, "#fee"],
        hover_data=["Name"],
        trendline="ols",
        width=480,
        height=480,
        template="simple_white",
        title=(
            f"Relationship between Rank and {column}<br>"
            f"<sup>Correlation: {correlation_with_rank[column]:.4f}</sup>"
        ),
    )
    fig.update_layout(
        font_family="serif", margin=dict(l=0, r=0, b=0), title_font_size=18, title_x=0.5
    )
    fig.update_xaxes(fixedrange=True)
    fig.update_yaxes(fixedrange=True)
    fig.show()